In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub
%pip install accelerate bitsandbytes faiss-cpu pypdf sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("/content/drive/MyDrive/RAG/Academy_Awards.pdf")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, separators=[
        "\n\n",
        ".",
        "\n",
        " ",
        "",
    ])
all_splits = text_splitter.split_documents(data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/RAG
%mkdir faissdb

/content/drive/MyDrive/RAG
mkdir: cannot create directory ‘chromadb’: File exists


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary specifying CPU as model configuration for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
hf = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(all_splits, hf)

# Persist the vectors locally on disk
vectorstore.save_local("/content/drive/MyDrive/RAG/faissdb")

# Load from local storage
# vectorstore = FAISS.load_local("/content/drive/MyDrive/RAG/chroma-db", hf, allow_dangerous_deserialization=True)

In [ ]:
question = "Who won the Academy award for best actor in 2023?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [ ]:
for doc in docs:
  print(doc)
  print('/n/n')

page_content='. Retrieved March 14, 2024.\n"Will Smith isn\'t the only actor to cause consternation at the world\'s most coveted film\nawards."\nStarkey, Adam (January 23, 2023). "Who has won the most Oscars?" (https://www.nme.c\nom/news/film/who-won-most-oscars-3382332). NME. Archived (https://web.archive.org/w\neb/20230124013232/https://www.nme.com/news/film/who-won-most-oscars-3382332)\nfrom the original on January 24, 2023. Retrieved March 14, 2024' metadata={'source': '/content/drive/MyDrive/RAG/Academy_Awards.pdf', 'page': 33}
/n/n
page_content='.[9][10]\nThe first Best Actor awarded was Emil Jannings, for\nhis performances in The Last Command and The\nWay of All Flesh. He had to return to Europe before\nthe ceremony, so the Academy agreed to give him\nthe prize earlier; this made him the first Academy\nAward winner in history' metadata={'source': '/content/drive/MyDrive/RAG/Academy_Awards.pdf', 'page': 1}
/n/n
page_content='. Retrieved March 14,\n2024. "And are the Oscars, given

In [ ]:
# Creating retriever from the vectorstore with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [ ]:
docs = retriever.get_relevant_documents("Who won the Academy award for best actor in 2023?")

In [ ]:
for doc in docs:
  print(doc)



## LLM using HuggingFace


In [ ]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig


In [ ]:
model_id = "pankajmathur/orca_mini_3b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                 model_id,
                 device_map="auto",
                 load_in_8bit=True
                 )
pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
                max_new_tokens=1024
               )

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
llm.invoke('Who won the Academy award for best actor in 2024?')

''' Output:
Who won the Academy award for best actor in 2024?
Answer: Will Smith won the Academy award for best actor in 2024.'''

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'Who won the Academy award for best actor in 2024?\nAnswer: Will Smith won the Academy award for best actor in 2024.'

## Retrieval QA

In [ ]:
from langchain.chains import RetrievalQA
from langchain import hub

# Loads the latest version of the RAG prompt
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": prompt}, chain_type="stuff"
)

In [ ]:
question = "Who won the Academy award for best actor in 2024?"
result = qa_chain.run({"query": question})
print(result)

''' Output:
Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Who won the Academy award for best actor in 2024?
Context: .[9][10]
The first Best Actor awarded was Emil Jannings, for
his performances in The Last Command and The
Way of All Flesh. He had to return to Europe before
the ceremony, so the Academy agreed to give him
the prize earlier; this made him the first Academy
Award winner in history

. Retrieved March 14,
2024. "And are the Oscars, given out by Hollywood's most prestigious professional
association, the biggest prize in the world — or just in America?"

. Retrieved March 14, 2024.
"Will Smith isn't the only actor to cause consternation at the world's most coveted film
awards."
Starkey, Adam (January 23, 2023). "Who has won the most Oscars?" (https://www.nme.c
om/news/film/who-won-most-oscars-3382332). NME. Archived (https://web.archive.org/w
eb/20230124013232/https://www.nme.com/news/film/who-won-most-oscars-3382332)
from the original on January 24, 2023. Retrieved March 14, 2024

Best in films in 2023

Award Best Actor Best Actress
Winner Cillian Murphy
(Oppenheimer)Emma Stone
(Poor Things)

Award Best Supporting
ActorBest Supporting
Actress
WinnerRobert Downey Jr
Answer: Cillian Murphy won the Best Actor award in 2024, and Emma Stone won the Best Actress award. '''

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Who won the Academy award for best actor in 2024? 
Context: .[9][10]
The first Best Actor awarded was Emil Jannings, for
his performances in The Last Command and The
Way of All Flesh. He had to return to Europe before
the ceremony, so the Academy agreed to give him
the prize earlier; this made him the first Academy
Award winner in history

. Retrieved March 14,
2024. "And are the Oscars, given out by Hollywood's most prestigious professional
association, the biggest prize in the world — or just in America?"

. Retrieved March 14, 2024.
"Will Smith isn't the only actor to cause consternation at the world's most coveted film
awards."
Starkey, Adam (January 23, 2023). "Who has won the most Oscars?" (https://www.nme.c
om/news/film/who-won-

' Output:\nHuman: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\nQuestion: Who won the Academy award for best actor in 2024?\nContext: .[9][10]\nThe first Best Actor awarded was Emil Jannings, for\nhis performances in The Last Command and The\nWay of All Flesh. He had to return to Europe before\nthe ceremony, so the Academy agreed to give him\nthe prize earlier; this made him the first Academy\nAward winner in history\n\n. Retrieved March 14,\n2024. "And are the Oscars, given out by Hollywood\'s most prestigious professional\nassociation, the biggest prize in the world — or just in America?"\n\n. Retrieved March 14, 2024.\n"Will Smith isn\'t the only actor to cause consternation at the world\'s most coveted film\nawards."\nStarkey, Adam (January 23, 2023). "Who has won the most Oscars?" (https:/

In [ ]:
question = "What was my last question?"
result = qa_chain.run({"query": question})
print(result)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: What was my last question? 
Context: Going My Way
Wilson
Roman Holiday
Giant
Sayonara
The Apartment
Lawrence of Arabia
Tom Jones
The Sound of Music
Doctor Zhivago
Bonnie and Clyde
Guess Who's Coming to Dinner
Anne of the Thousand Days
PattonForrest Gump
Chicago
The Hurt Locker
Mad Max: Fury Road
La La Land
Dune
5It Happened One Night
How Green Was My Valley
Wilson
The Bad and the Beautiful
Around the World in 80 Days
The King and I
The Apartment
Mary Poppins
The Sound of Music
Doctor Zhivago
Who's Afraid of Virginia Woolf?

.[81] This segment has drawn criticism over the years for the omission of some names.
Criticism was also levied for many years regarding another aspect, with the segment having a "popul arity
contest" feel as the au

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

chat_qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
question = "Who won the Academy Award for best actor in 2024?"
result = chat_qa({"question": question})
print(result['answer'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

.[9][10]
The first Best Actor awarded was Emil Jannings, for
his performances in The Last Command and The
Way of All Flesh. He had to return to Europe before
the ceremony, so the Academy agreed to give him
the prize earlier; this made him the first Academy
Award winner in history

. Retrieved March 14,
2024. "And are the Oscars, given out by Hollywood's most prestigious professional
association, the biggest prize in the world — or just in America?"

. Retrieved March 14, 2024.
"Will Smith isn't the only actor to cause consternation at the world's most coveted film
awards."
Starkey, Adam (January 23, 2023). "Who has won the most Oscars?" (https://www.nme.c
om/news/film/who-won-most-oscars-3382332). NME. Archived (https://web.archive.org/w
eb/20230124013232/https://www.nme.com/news/film/who-won-most-oscars-3382332)
from the or

In [ ]:
question = "For which film did he win the award?"
result = chat_qa({"question": question})
print(result)

{'question': 'For which film did he win the award?', 'chat_history': [HumanMessage(content='Who won the Academy Award for best actor in 2024?'), AIMessage(content='Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\n.[9][10]\nThe first Best Actor awarded was Emil Jannings, for\nhis performances in The Last Command and The\nWay of All Flesh. He had to return to Europe before\nthe ceremony, so the Academy agreed to give him\nthe prize earlier; this made him the first Academy\nAward winner in history\n\n. Retrieved March 14,\n2024. "And are the Oscars, given out by Hollywood\'s most prestigious professional\nassociation, the biggest prize in the world — or just in America?"\n\n. Retrieved March 14, 2024.\n"Will Smith isn\'t the only actor to cause consternation at the world\'s most coveted film\nawards."\nStarkey, Adam (January 23, 2023). "Who has won the most Oscars?" (htt

In [ ]:
print(result['answer'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

.[9][10]
The first Best Actor awarded was Emil Jannings, for
his performances in The Last Command and The
Way of All Flesh. He had to return to Europe before
the ceremony, so the Academy agreed to give him
the prize earlier; this made him the first Academy
Award winner in history

. Retrieved March 14, 2024.
"Will Smith isn't the only actor to cause consternation at the world's most coveted film
awards."
Starkey, Adam (January 23, 2023). "Who has won the most Oscars?" (https://www.nme.c
om/news/film/who-won-most-oscars-3382332). NME. Archived (https://web.archive.org/w
eb/20230124013232/https://www.nme.com/news/film/who-won-most-oscars-3382332)
from the original on January 24, 2023. Retrieved March 14, 2024

. Retrieved March 14,
2024. "And are the Oscars, given out by Hollywood's most prestigious professional
association, t